In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pybdshadow

# 数据读取

In [6]:
buildings = gpd.read_file(r'../example/data/Suzhoubuildings/苏州(1).shp')
buildings = pybdshadow.bd_preprocess(buildings)

In [2]:
buildings = gpd.read_file(r'../example/data/bd_demo.json')
buildings = pybdshadow.bd_preprocess(buildings)

# 平行光阴影

In [8]:
#sunlight
date = pd.to_datetime('2015-01-01 02:45:33.959797119')
#shadows = pybdshadow.bdshadow_sunlight(buildings, date,merge = True)

# 点光源阴影

In [5]:
#pointlight
shadows = pybdshadow.bdshadow_pointlight(buildings,139.713319,35.552040,100)

# 可视化建筑与阴影

In [6]:
pybdshadow.show_bdshadow(buildings = buildings,shadows = shadows)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '4eo0v3', 'type': '…

# 广告可视范围计算

In [10]:
#广告
adp1 = np.array([139.713319,35.552040])
adp2 = np.array([139.711861,35.553646])
orientation = pybdshadow.calOrientation(adp1,adp2)
brandCenter = (adp1+adp2)/2
pybdshadow.initialVisualRange(brandCenter,orientation)

IndexError: index 2 is out of bounds for axis 0 with size 2